# Automated Cv Macthing Phase One

In [20]:
from langchain_pinecone import PineconeVectorStore
from pydantic import BaseModel
import instructor
from openai import OpenAI
import requests
from langchain.indexes import SQLRecordManager,index
from langchain_community.document_loaders import (
    PyPDFLoader,
    S3FileLoader,
    S3DirectoryLoader,
    Docx2txtLoader
    )
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
import os
import pathlib

os.environ['PINECONE_API_KEY'] = ''
api_keys = ''

index_name = "job-descriptions"

embeddings =OpenAIEmbeddings(api_key=api_keys)
text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=6000,
        chunk_overlap=500,
        length_function=len,
        is_separator_regex=False,
)


In [ ]:
### file loader func
## Phase 1 (one time process)



def embed_load_document_to_pincone(file_path): 
    """
    load job description to pinecone
    check for file extension
    if pdf load pdf loader
    if docx load docx loader
    
    """ 
    ## initializing langchain indexing
    source = 'resume'
    namespace = f'pincone-{source}'
    record_manager = SQLRecordManager(namespace, db_url=f'sqlite:///resume.sql')
    record_manager.create_schema()
    
    
    ## checking file_path extension
    file_extention = pathlib.Path(file_path).suffix
    if file_extention == ".pdf":
        pdf_loader =  PyPDFLoader(file_path)
        doc = text_splitter.split_documents(pdf_loader.load())
    else:
        word_loader = Docx2txtLoader(file_path)
        doc = text_splitter.split_documents(word_loader.load())
        
        
    ## indexing document Avoid writing duplicated content into the vector store
    store = PineconeVectorStore(text_key='text',embedding=embeddings, index_name=index_name)
    index(doc, record_manager, store, cleanup="incremental", source_id_key='source')
    
        
    for docs in doc : print(docs.page_content) 
    
   
    ##  loading document to pincone vector 
    vectostore = PineconeVectorStore.from_documents(doc, embeddings, index_name=index_name)
    
    
    print("document uploaded to pincone db.........")
    
embed_load_document_to_pincone("samplefile/wordJobDescription.docx")

## Automated Cv Macthing Phase two

In [32]:
### file loader extractor func
## Phase 2 (batch and realtime)

def resume_loader(storage_name=None, localfile=None):
  
    ## checking storage type s3 or dropbox
    if storage_name == 's3':
      s3_loader = S3DirectoryLoader(
        'resume-bucket',
        prefix="resume",
        aws_access_key_id="", 
        aws_secret_access_key=""
        )
      doc = text_splitter.split_documents(s3_loader.load())
      return doc
    
    elif storage_name == 'dropbox':
        pass
    else:
      
       ## checking file_path extension
      if localfile:
        file_extention = pathlib.Path(localfile).suffix
        if file_extention == ".pdf":
            pdf_loader =  PyPDFLoader(localfile)
            doc = text_splitter.split_documents(pdf_loader.load())
            
        else:
            word_loader = Docx2txtLoader(localfile)
            doc = text_splitter.split_documents(word_loader.load())
          
        #for docs in doc : print(docs.page_content) 
        return doc
          
resume_loaders = resume_loader(localfile='samplefile/Owolabi_Akintan_resume.pdf')





In [33]:
 ## Similarity search 

client = instructor.from_openai(OpenAI(api_key=api_keys))


class MatchResume(BaseModel):
    """
    instructor response_model class
    
    """
    resume: str 
    

def job_description_augment_prompt_match(resume_query):
    
    # get top 3 results from knowledge base
    
    vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings)
    results = vectorstore.similarity_search_with_relevance_scores(resume_query, k=3)
    
    ## retrive result and score and perform rag process using instructor  
    for result, score in results:
        
        source_knowledge = f"Document: {result.page_content},\n Score: {score}"
        
        resp = client.chat.completions.create(
            model="gpt-3.5-turbo",
            response_model=MatchResume,
            messages=[
                {"role": "system", "content": "You are a resume job description matcher. If Resume does not match the job description, return 'No Match'."},
                {"role": "user", "content": f"using the Contexts: Job Description: \n {str(source_knowledge)} \n  match the following Resumes : {resume_query}"}
            ]
        )
     

    return resp.model_dump() 


In [34]:
# Webhook api


def webhook_api(payload):
    header = {'Content-Type':'application/json',
              'X-Api-Key':'ef9f9fb91d819f7fae0fda9da47b11116e28564c49cd60d2017917ca99b6c3f4'}
    url = "https://sapphire.contextdata.ai/api/webpush/c5-000001uhj6bd/"
    response = requests.post(url,json=payload,headers=header)
    return response.status_code


## send match resume to webhook api endpoint
for don in resume_loaders:
    matcher_resume_payload = job_description_augment_prompt_match(don.page_content[:]) 
    print("uploading match resume to webhook payload")
    webhook_api(matcher_resume_payload)
    

uploading match resume to webhook payload
uploading match resume to webhook payload
